In [1]:
#Notebook ile Drive arasındaki bağlantıyı sağlayalım
#Erişimi uzun olan drive dizini için kısa bir yol tanıma yapıyoruz
# "/content/gdrive/My\ Drive/" olan dizine /mydrive diyerek ulaşabiliriz.
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/

/
Mounted at /content/gdrive


In [4]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from PIL import Image
import tensorflow as tf
from pathlib import Path
from tensorflow import  keras
from keras import layers
from keras.models import Sequential, Model
import pickle
import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import random
import glob
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras_preprocessing.image.utils import load_img
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
#!pip install keras_preprocessing

In [ ]:
#!nvidia-smi

!pip install keras_preprocessing

In [5]:
batch_size = 32
target_size=(256,256)

In [ ]:
d_yolu='/content/gdrive/MyDrive/AiResim/test'

'/content/gdrive/MyDrive/AiResim/test'

In [ ]:
# Veri seti klasörünün yolu (Data klasörünüzün yolu)
data_dir = Path(d_yolu)
filepaths = list(data_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
images = pd.concat([filepaths, labels], axis=1)
train_df, test_df = train_test_split(images, train_size=0.8, shuffle=True, random_state=42)

In [ ]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
        validation_split=0.2,
        vertical_flip=True,
        rescale=1./255,
        rotation_range=30,  # Örneğin, 0 ile 90 derece arasında rastgele döndürme
        zoom_range=[0.8, 1.2]  # Rastgele yakınlaştırma aralığı
) # %70 eğitim datası içinden ön eğitim kontrölü içinde %20 yani 140 adet ön test (daha önceden gördüğü veri üzerinden test yapmak için) veri ayırıyoruz

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input,)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    labels='inferred',
    label_mode='int',
    y_col='Label',
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=123,
    cros_to_aspect_ratio =False,
    follow_links=False,
    interpolation='bilinear',
    subset='training'
)
val_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    label_mode='int',
    labels='inferred',
    y_col='Label',
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    interpolation='bilinear',
    shuffle=True,
    seed=123,
    follow_links=False,
    cros_to_aspect_ratio =False,
    subset='validation'
)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    label_mode='int',
    labels='inferred',
    y_col='Label',
    target_size=target_size,
    color_mode='rgb',
    class_mode='categorical',
    interpolation='bilinear',
    batch_size=batch_size,
    cros_to_aspect_ratio =False,
    follow_links=False,
    shuffle=False
)

Found 896 validated image filenames belonging to 7 classes.
Found 224 validated image filenames belonging to 7 classes.
Found 280 validated image filenames belonging to 7 classes.


In [ ]:
CLASNAME=7

In [ ]:
# Load VGG16 model
_model_vgg16 = tf.keras.applications.vgg16.VGG16(
    include_top=False,
    input_shape=(256, 256, 3),
    weights="imagenet"
)

# Freeze VGG16 layers
for layer in _model_vgg16.layers:
    layer.trainable = False

# Create your custom model on top of VGG16
x = _model_vgg16.output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation="relu")(x)
x = tf.keras.layers.Dense(50, activation='relu')(x)
output = tf.keras.layers.Dense(CLASNAME, activation="softmax")(x)

model = tf.keras.Model(inputs=_model_vgg16.input, outputs=output)

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()

In [ ]:
# Adam Optimizer used for training
optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0003,
    name="Adam",
)

In [ ]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath="/content/gdrive/MyDrive/AiResim/model_data_VGG_checkpoints",
        monitor="val_loss",
        verbose=1,
        save_best_only=True,
        save_weights_only=False,
        mode="min",
        save_freq="epoch"
    )
]
# Modeli derlerken callbacks parametresine callbacks listesini ekleyin
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'], callbacks=callbacks)

# Modeli eğitirken callbacks parametresine callbacks listesini ekleyin
model.fit(train_generator,validation_data=val_generator,epochs=50,callbacks=[callbacks])


from keras.optimizers import Adam
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[callbacks])

In [ ]:
class_names = test_generator.class_indices
class_num =len(class_names)
print(class_names, class_num)

{'Aktivite': 0, 'Çocuk': 1, 'Dış Görünüm': 2, 'Havuz-Plaj': 3, 'Odalar': 4, 'Toplantı Salonu': 5, 'Yeme-İçme': 6} 7


In [ ]:
results = model.evaluate(val_generator, verbose=0)
print(f"Test Accuracy: {np.round(results[1] * 100,2)}%")

Test Accuracy: 65.18%


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

predec = np.argmax(model.predict(test_generator),axis=1)
matrix = confusion_matrix(test_generator.labels,predec)
report = classification_report(test_generator.labels,predec,target_names=test_generator.class_indices,zero_division=0)


18/18 [==============================] - 152s 9s/step


In [ ]:
print("rapor",report)

In [ ]:
model.save("/content/gdrive/MyDrive/AiResim/22_Mart_vgg16.keras") #h5 yeni sürüm .keras

In [ ]:
'''

import warnings
from keras.models import load_model
from tensorflow.keras.preprocessing import image
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import shutil  # Added import for moving files

# Test Dosyası
dir_path = '/content/gdrive/MyDrive/AiResim/Source/testing'

# Modeli yükle
model = load_model('/content/gdrive/MyDrive/AiResim/19_Mart_vgg16.keras')

# Sınıf etiketlerini içeren bir sözlük
class_labels = {'Aktivite': '0', 'Çocuk': '1', 'Dış Görünüm': '2', 'Havuz-Plaj': '3', 'Odalar': '4', 'Toplantı Salonu': '5', 'Yeme-İçme': '6'}

# Sınıf etiketlerini içeren bir DataFrame oluştur
df = pd.DataFrame({'names': list(class_labels.keys()), 'index': list(class_labels.values())})

# Yeni bir 'output' klasörü oluştur
#output_folder = 'C:/Users/info/OneDrive/Masaüstü/alt image/output_4'
#os.makedirs(output_folder, exist_ok=True)

for i in os.listdir(dir_path):
    img_path = os.path.join(dir_path, i)
    img = image.load_img(img_path, target_size=(256, 256))
    plt.imshow(img)
    plt.show()

    X = image.img_to_array(img)
    X = np.expand_dims(X, axis=0)
    images = np.vstack([X])
    predictions = np.argmax(model.predict(images), axis=1)

    # Tahmin edilen sınıf etiketini sınıf numarası ile ekrana yazdır
    predicted_class = predictions[0] + 1  # Sınıflar 1'den başladığı için 1 eklenir
    print("\n", predicted_class)

    # Oluşturulan sınıf klasörüne görseli taşı
    #class_folder = os.path.join(output_folder, str(predicted_class))
    #os.makedirs(class_folder, exist_ok=True)

    # Yeni klasöre görseli taşı
    #shutil.move(img_path, os.path.join(class_folder, i))

'''